In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python -m textblob.download_corpora
!pip install pymorphy2

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 13.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=7fd0789ea6684dae2bdec364b8

In [3]:
import pandas as pd
from textblob import TextBlob, Word

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import pymorphy2


# nltk.download('omw-1.4')
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.

True

**Exploratory analysis**

In [4]:
f_data = '/content/drive/MyDrive/test_data.csv'
data = pd.read_csv(f_data)
data.iloc[250:270, :]

,dlg_id,line_n,role,text
250,3,1,manager,Алло дмитрий добрый день
251,3,2,manager,Добрый меня максим зовут компания китобизнес у...
252,3,3,client,Да удобно
253,3,4,manager,Да дмитрий вот мне моя коллега анастасия подск...
254,3,5,client,А задача в общем то
255,3,6,client,Стоит из чего сейчас секунду
256,3,7,manager,Угу
257,3,8,client,В общем
258,3,9,client,Настроить воронки как бы и все такое я могу сам
259,3,10,client,Вот и даже сделать этого сэйл с ботом


In [5]:
# Выборка реплик менеджера из данных и подсчет числа диалогов в наборе данных

m_data = data[data['role']=='manager']
# m_data.groupby(['dlg_id']).count()
dlg_count = pd.unique(m_data['dlg_id'])
dlg_count

array([0, 1, 2, 3, 4, 5])

In [6]:
# Объединение всех реплик менеджера в единый текст

data_txt = m_data.groupby(['dlg_id'])['text'].transform(lambda x: '. '.join(x)).drop_duplicates().reset_index()
data_txt['text'] = data_txt['text'].transform(lambda x: x.lower())

data_txt

,index,text
0,1,алло здравствуйте. меня зовут ангелина компани...
1,110,алло здравствуйте. меня зовут ангелина компани...
2,166,алло здравствуйте. меня зовут ангелина компани...
3,250,алло дмитрий добрый день. добрый меня максим з...
4,305,вот по виду платежи пообщаться помните мы вот ...
5,338,да это анастасия. угу. так да да хорошо смотри...


In [ ]:
# blob = TextBlob(data_txt['text'][5])
# blob.sentences
# blob.noun_phrases
# blob.ngrams()[:5]

1

In [7]:
# Создание для каждого текста менеджера объекта TextBlob

bls = [TextBlob(data_txt['text'][i]) for i in dlg_count]
# bls
bls[4].ngrams()
bls[2].sentences

[Sentence("алло здравствуйте."),
 Sentence("меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там."),
 Sentence("а нет я еще не отправляла я решила вам набрать."),
 Sentence("удобно вам сейчас говорить что я вам так рассказала."),
 Sentence("угу хорошо а вы сейчас подскажите по каким критериям выбираете партнера чтобы мы могли подготовить для вас предложение."),
 Sentence("угу я поняла ну по стоимости везде одинаковые по официальным партнерам."),
 Sentence("поэтому могу сказать вот по преимуществам то есть если про преимущества то у нас более 20 виджетов вам идет бесплатно а там же идет кэшбэк 20% вам эта сумма возвращается то есть когда то что вы можете."),
 Sentence("диджитал бизнес."),
 Sentence("здесь же это у бизнес."),
 Sentence("я вам еще раз говорю на ватсап предложение."),
 Sentence("вот и смотрите на счет кэшбэка то что у вас кэшбэк 20% вы за эту сумму можете например приобрести какой то бизнес процес

**Извлечение приветствия и прощания**

Предполагается, в компании задан формальный набор приветствий и прощаний, которые должны встречаться в репликах менеджера (они заданы, а не скачиваются из NLP-библиотек)

In [ ]:
greetings = {'здравствуйте', 'добрый день', 'добрый вечер', 'доброе утро'}
farewells = {'до свидания', 'всего доброго', 'всего хорошего'}

def find_greets_and_farws(bls):
  greet_replica = {}
  farw_replica = {}

  for i_idx, i in enumerate(bls):
    # for j in i.ngrams():  # Если требуется проверка объемного формального приветствия, можно увеличить размер нграммов
    #   print(' '.join(j))

    for j in i.sentences:
      # print(j[:-1])
      # print(set(j.split()))
      # greet = list(set(j[:-1].split()) & greetings)
      # farw = list(set(j[:-1].split()) & farewells)
      # print(greet)
      greet = [k for k in greetings if k in j]
      farw = [k for k in farewells if k in j]
      if greet:
        greet_replica[i_idx] = j
      if farw:
        farw_replica[i_idx] = j

  return greet_replica, farw_replica

find_greets_and_farws(bls)

({0: Sentence("алло здравствуйте."),
  1: Sentence("алло здравствуйте."),
  2: Sentence("алло здравствуйте."),
  3: Sentence("алло дмитрий добрый день.")},
 {0: Sentence("всего хорошего до свидания"),
  1: Sentence("до свидания"),
  3: Sentence("угу все хорошо да понедельника тогда всего доброго"),
  4: Sentence("во вторник все ну с вами да тогда до вторника до свидания"),
  5: Sentence("ну до свидания хорошего вечера")})

*Примечание: в задании "Извлекать реплики, где менеджер представил себя" не очевидно, что считать представлением менеджера, считается ли реплика менеджера в ответ на входящий звонок "это Анастасия" представлением*

**Проверка, поздоровался и попрощался ли менеджер**

In [ ]:
def check_greet_and_farw(greet_replica, farw_replica):
  return set(greet_replica.keys()) & set(farw_replica.keys())

{0, 1, 3}

**Извлечение представления менеджера**

In [ ]:
# Получение имен из диалогов

p_thresh = 0.5
morph = pymorphy2.MorphAnalyzer()

names =[]

for speech in data_txt["text"]:
    names_ = []
    for idx, word in enumerate(nltk.word_tokenize(speech)):
        # print(word)
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= p_thresh:
                print('{:<12}\t({:>12})\tscore:\t{:0.3}'.format(word, p.normal_form, p.score))
                names_.append(word)
    names.append(names_)
    print()
names

ангелина    	(    ангелина)	score:	1.0

ангелина    	(    ангелина)	score:	1.0

ангелина    	(    ангелина)	score:	1.0

дмитрий     	(     дмитрий)	score:	0.986
максим      	(      максим)	score:	0.636
дмитрий     	(     дмитрий)	score:	0.986
анастасия   	(   анастасия)	score:	0.939


анастасия   	(   анастасия)	score:	0.939
вячеслав    	(    вячеслав)	score:	0.882
максим      	(      максим)	score:	0.636
дмитрий     	(     дмитрий)	score:	0.986



[['ангелина'],
 ['ангелина'],
 ['ангелина'],
 ['дмитрий', 'максим', 'дмитрий', 'анастасия'],
 [],
 ['анастасия', 'вячеслав', 'максим', 'дмитрий']]

In [ ]:
# Менеджер представился + имя менеджера

intro_words = {'зовут', 'имя', 'это'}  # меня зовут, мое имя, это <имя>

managers = {}
intro_replica = {}
company_sentences = {}

for i_idx, i in enumerate(bls):
  for j in i.ngrams():  # по умолчанию 3 слова
    man_name = list(set(j) & set(names[i_idx]))
    is_intro_word = list(set(j) & intro_words)
    if is_intro_word and man_name:
      managers[i_idx] = man_name[0]
  for j in i.sentences:
    if i_idx in intro_replica.keys() and i_idx in company_sentences.keys():
      break
    if i_idx in managers.keys() and managers[i_idx] in j:
      intro_replica[i_idx] = j
    if 'компания' in j:
      company_sentences[i_idx] = j
# managers
# intro_replica
company_sentences

{0: Sentence("меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается."),
 1: Sentence("меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается."),
 2: Sentence("меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там."),
 3: Sentence("добрый меня максим зовут компания китобизнес удобно говорить.")}

**Извлечение названий компаний**

In [ ]:
stop_words = stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()
# speech_part = str(morph.parse('зовут')[0].tag).split(',')[0]

In [ ]:
# def exctract_companies(company_sentences):
sp_parts_in_company_name = ['NOUN', 'ADJF']

sentences = {idx: nltk.word_tokenize(str(i)[:-1]) for idx, i in company_sentences.items()}
sp_parts = {}
companies = {}

for idx, i in sentences.items():
  tokens = [s for s in i if s not in stop_words]
  sp_parts[idx] = {t: str(morph.parse(t)[0].tag).split(',')[0] for t in tokens}
  comp_name = ''
  k = list(sp_parts[idx].keys())
  v = list(sp_parts[idx].values())
  co_idx = k.index('компания')
  k = k[co_idx + 1:]
  v = v[co_idx + 1:]
  for c in zip(k, v):
    if c[1] in sp_parts_in_company_name:
      comp_name = comp_name + c[0] + ' '
    else:
      break

  companies[idx] = comp_name[:-1]
  
companies

{0: 'диджитал бизнес',
 1: 'диджитал бизнес',
 2: 'диджитал бизнес',
 3: 'китобизнес'}

In [ ]:
# sentences = nltk.word_tokenize(str(company_sentences[0])[:-1])

# print(nltk.pos_tag(nltk.word_tokenize(sent)))

tokens = [s for s in sentences if s not in stop_words]
sp_parts = {t: str(morph.parse(t)[0].tag).split(',')[0] for t in tokens}
sp_parts

{'зовут': 'VERB',
 'ангелина': 'NOUN',
 'компания': 'NOUN',
 'диджитал': 'ADJF',
 'бизнес': 'NOUN',
 'звоним': 'VERB',
 'поводу': 'NOUN',
 'продления': 'NOUN',
 'лицензии': 'NOUN',
 'серым': 'ADJF',
 'скоро': 'ADVB',
 'срок': 'NOUN',
 'заканчивается': 'VERB'}